In [45]:
import numpy as np
# A = 13
# B = 180
# C = 17
# D = 90
# E = 6
# F = 80
# G = 280
# H = 80
# J = 50
# t = 200

A = int(input("Enter the value of Resistance in Z1(in ohms): "))
B = int(input("Enter the value of Inductance in Z1(in mH): "))
C = int(input("Enter the value of Resistance in Z2(in ohms): "))
D = int(input("Enter the value of Capacitance in Z2(in uF):"))
E = int(input("Enter the value of Resistance in Z3(in ohms): "))
F = int(input("Enter the value of Inductance in Z3(in mH): "))
G = int(input("Enter the voltage value: "))
H = int(input("Enter the phase of the voltage(in degrees): "))
J = int(input("Enter the frequency of the voltage(in Hz): "))
t = int(input("Enter the time for calculating the energy (in mins)"))

![Circuit Diagram](image4.png)

The impedance $Z_1$ consists of a resistance of $A \, \Omega$, and an inductance of $B \, \text{mH}$.
The impedance $Z_2$ consists of a resistance of $C \, \Omega$, and a capacitance of $D \, \mu\text{F}$.
The impedance $Z_3$ consists of a resistance of $E \, \Omega$, and an inductance of $F \, \text{mH}$.

The voltage source is $G \angle H^\circ \, \text{V}$ at $J \, \text{Hz}$.


In [66]:
# Question 1
def tot_imp_in_rec_coord(A,B,C,D,E,F,G,H,J):   
    X_L_1 = 2*np.pi*J*B*1e-3
    X_L_2 = 2*np.pi*J*F*1e-3
    X_C = 1/(2*np.pi*J*D*1e-6)
    deno = ((C+E)**2+(X_C-X_L_2)**2)
    a = ((C*E+X_C*X_L_2)*(C+E) + (C*X_L_2-E*X_C)*(X_L_2-X_C))/deno + A
    b = ((C*X_L_2-E*X_C)*(C+E) - (X_L_2-X_C)*(C*E+X_C*X_L_2))/deno + X_L_1
    return(a,b)
# DISPLAYED IN (A,B) FOR A + iB

In [67]:
a,b = tot_imp_in_rec_coord(A,B,C,D,E,F,G,H,J)
print(complex(a,b))

(45.48791606369803+80.35582461520676j)


In [68]:
# Question 2
def curr_I1(A,B,C,D,E,F,G,H,J):
    a,b = tot_imp_in_rec_coord(A,B,C,D,E,F,G,H,J)
    angle = H*np.pi/180
    coeff = (G/(a**2 + b**2))
    c = coeff*(a*np.cos(angle) + b*np.sin(angle))
    d = coeff*(a*np.sin(angle) - b*np.cos(angle))
    return (c,d)

In [69]:
c,d = curr_I1(A,B,C,D,E,F,G,H,J)
print(complex(c,d))

(2.858187152463388+1.0128884611489803j)


In [70]:
# Question 3
def vol_across_Z2(A,B,C,D,E,F,G,H,J):
    angle = H*np.pi/180
    X_L_1 = 2*np.pi*J*B*1e-3
    
    c,d = curr_I1(A,B,C,D,E,F,G,H,J)
    e = G*np.cos(angle) + (d*X_L_1-A*c)
    f = (G*np.sin(angle)-X_L_1*c - A*d)
    return (e,f)

def mag_vol(A,B,C,D,E,F,G,H,J):
    e,f = vol_across_Z2(A,B,C,D,E,F,G,H,J)
    return np.sqrt(e**2+f**2)

In [71]:
print(mag_vol(A,B,C,D,E,F,G,H,J))

122.13448893983835


In [72]:
# Question 4
def power_factor(A,B,C,D,E,F,G,H,J):
    a,b = tot_imp_in_rec_coord(A,B,C,D,E,F,G,H,J)
    return np.cos(np.arctan(b/a))
# if negative then the voltage is lagging, else leading
# if negative then the current if leading, else lagging

In [73]:
print(power_factor(A,B,C,D,E,F,G,H,J))

0.49262682006931596


In [74]:
# Question 5
def pow_fac_equal_1(A,B,C,D,E,F,G,H,J):
    a,b = tot_imp_in_rec_coord(A,B,C,D,E,F,G,H,J)
    X = -(a**2+b**2)/b
    if(X < 0):
        X = abs(X)
        print("Capacitance")
        return 1e6/(2*np.pi*J*X)
    else:
        print("Inductance")
        return 1e3*X/2*np.pi*J
# if negative then capacitive, else inductive

In [75]:
print(pow_fac_equal_1(A,B,C,D,E,F,G,H,J))

Capacitance
29.99932703414994


In [76]:
# Question 6
def phase_ang_diff(A,B,C,D,E,F,G,H,J):
    X_C = 1/(2*np.pi*J*D*1e-6)
    phase = np.arctan(X_C/C)
    return phase*180/np.pi

# if negative then the voltage is leading, else lagging
    

In [77]:
print("Phase of I2 - phase of V3: ",phase_ang_diff(A,B,C,D,E,F,G,H,J))

Phase of I2 - phase of V3:  64.328096916864


In [78]:
# Question 7
def power_loss_percentage(A,B,C,D,E,F,G,H,J):
    X_C = 1/(2*np.pi*J*D*1e-6)
    X_L_2 = 2*np.pi*J*F*1e-3
    a,b = tot_imp_in_rec_coord(A,B,C,D,E,F,G,H,J)
    c,d = curr_I1(A,B,C,D,E,F,G,H,J)
    e,f = vol_across_Z2(A,B,C,D,E,F,G,H,J)
    coeff1 = C**2 + X_C**2
    coeff2 = E**2 + X_L_2**2
    m = (e*C - f*X_C)/coeff1
    n = (f*C + e*X_C)/coeff1
    p = (e*E + f*X_L_2)/coeff2
    q = (f*E - e*X_L_2)/coeff2
    
    return (m**2+n**2)*C*100/((c**2+d**2)*A + (m**2+n**2)*C + (p**2+q**2)*E)

In [79]:
print(power_loss_percentage(A,B,C,D,E,F,G,H,J))

39.371584288216596


In [80]:
# Question 8
def react_power(A,B,C,D,E,F,G,H,J):

    X_C = 1/(2*np.pi*J*D*1e-6)
    e,f = vol_across_Z2(A,B,C,D,E,F,G,H,J)
    coeff1 = C**2 + X_C**2
    m = (e*C - f*X_C)/coeff1
    n = (f*C + e*X_C)/coeff1
    return (m**2+n**2)*X_C

In [81]:
print(react_power(A,B,C,D,E,F,G,H,J))

342.60816905277403


In [82]:
# Question 9
def energy_consump(A,B,C,D,E,F,G,H,J,t):
    c,d = curr_I1(A,B,C,D,E,F,G,H,J)
    energy = (c**2+d**2)*A*t/(60000)
    return energy

In [83]:
print(energy_consump(A,B,C,D,E,F,G,H,J,t))

0.3984576627735393


In [84]:
# Question 10
def Vab_and_Vac(A,B,C,D,E,F,G,H,J):
    # Vab - Vac
    c,d = curr_I1(A,B,C,D,E,F,G,H,J)
    X_L_1 = 2*np.pi*J*B*1e-3
    if (((X_L_1*c+A*d)/(A*c-X_L_1*d)) < 0):
        return (180 - np.arctan(abs((X_L_1*c+A*d)/(A*c-X_L_1*d)))*180/np.pi - H)
    else:
        return (np.arctan(abs((X_L_1*c+A*d)/(A*c-X_L_1*d)))*180/np.pi - H)
    

In [85]:
print("Vab - Vac = ",Vab_and_Vac(A,B,C,D,E,F,G,H,J))

Vab - Vac =  16.566577302593686
